In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv(r"data\pre-processed_inequality_data.csv")

In [2]:
data = data.rename(columns={"Unnamed: 0": "Year"})

In [3]:
data = data.drop(["Unnamed: 7", "Unnamed: 8"], axis=1)

In [4]:
data["lnY"] = np.log(data["GDP per capita (constant prices)"])
data["lnHC"] = np.log(data["Human Capital Index"])
data["lnL"] = np.log(data["Population growth (annual percentage)"])
data["lnK"] = np.log(data["Capital stock at constant national prices"])
data["lnGINI"] = np.log(data["Gini coefficient"])
data["lnINF"] = np.log(data["Annual percentage change in consumer price index"])

c:\Anaconda3\envs\ds_experiments\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
data

,Year,GDP per capita (constant prices),Human Capital Index,Population growth (annual percentage),Capital stock at constant national prices,Gini coefficient,Annual percentage change in consumer price index,lnY,lnHC,lnL,lnK,lnGINI,lnINF
0,1961,15427.13180,2.571570,0.760460,2712982.50,0.261,4.196499,9.643883,0.944517,-0.273832,14.813559,-1.343235,1.434251
1,1962,15464.82661,2.589920,0.848661,2826336.00,0.248,2.018544,9.646323,0.951627,-0.164095,14.854492,-1.394327,0.702376
2,1963,16095.44214,2.608400,0.748366,2945149.00,0.271,3.281587,9.686291,0.958737,-0.289863,14.895670,-1.305636,1.188327
3,1964,16885.79277,2.627012,0.650258,3130285.25,0.264,4.773775,9.734228,0.965847,-0.430386,14.956635,-1.331806,1.563137
4,1965,17135.07448,2.645757,0.642469,3306248.25,0.251,3.909618,9.748883,0.972957,-0.442437,15.011325,-1.382302,1.363440
5,1966,17307.97522,2.664636,0.551303,3471607.75,0.261,2.482116,9.758923,0.980067,-0.595470,15.060128,-1.343235,0.909111
6,1967,17692.85840,2.683649,0.538544,3653941.50,0.251,4.697428,9.780916,0.987178,-0.618887,15.111317,-1.382302,1.547015
7,1968,18570.51844,2.702798,0.486768,3856676.25,0.250,5.446664,9.829331,0.994288,-0.719967,15.165316,-1.386294,1.695003
8,1969,18852.07565,2.722084,0.415803,4046189.00,0.257,6.366568,9.844378,1.001398,-0.877543,15.213286,-1.358679,1.851061
9,1970,19285.64976,2.741508,0.398722,4227484.00,0.259,9.444837,9.867117,1.008508,-0.919490,15.257118,-1.350927,2.245468


In [6]:
data = data.dropna()

In [7]:
import statsmodels.api as sm

# Prepare the feature matrix with two separate constants
X = data[['lnHC', 'lnL', 'lnK', 'lnGINI', 'lnINF']].copy()
X['const1'] = 1
X['const2'] = 1

# Define the dependent variable
y = data['lnY']

# Fit the model and print results
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    lnY   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     1855.
Date:                Fri, 07 Feb 2025   Prob (F-statistic):           1.40e-54
Time:                        10:15:26   Log-Likelihood:                 123.57
No. Observations:                  55   AIC:                            -235.1
Df Residuals:                      49   BIC:                            -223.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
lnHC           1.6964      0.446      3.802      0.0

In [8]:
from statsmodels.tsa.stattools import zivot_andrews

# Run Perron-like (Zivot-Andrews) tests on each relevant column in X
vars_to_test = ['lnHC', 'lnL', 'lnK', 'lnGINI', 'lnINF']
for var in vars_to_test:
    result = zivot_andrews(X[var].dropna(), maxlag=10, regression='c')
    print(f"Perron Test (Zivot-Andrews) for {var}:")
    print("  Test statistic:", result[0])
    print("  p-value:", result[1])
    print("  Break point (0-based index):", result[3])
    print()

# Run Perron-like (Zivot-Andrews) test on y
result_y = zivot_andrews(y.dropna(), maxlag=10, regression='c')
print("Perron Test (Zivot-Andrews) for y:")
print("  Test statistic:", result_y[0])
print("  p-value:", result_y[1])
print("  Break point (0-based index):", result_y[3])

Perron Test (Zivot-Andrews) for lnHC:
  Test statistic: -1.361958705955816
  p-value: 0.9990000000000001
  Break point (0-based index): 0

Perron Test (Zivot-Andrews) for lnL:
  Test statistic: -4.666658475474781
  p-value: 0.0772012898269238
  Break point (0-based index): 0

Perron Test (Zivot-Andrews) for lnK:
  Test statistic: -2.0619637062128455
  p-value: 0.9959600223902104
  Break point (0-based index): 0

Perron Test (Zivot-Andrews) for lnGINI:
  Test statistic: -4.390247696831279
  p-value: 0.1556487435856406
  Break point (0-based index): 5

Perron Test (Zivot-Andrews) for lnINF:
  Test statistic: -4.604153884127791
  p-value: 0.0913836712362065
  Break point (0-based index): 6

Perron Test (Zivot-Andrews) for y:
  Test statistic: -3.8230976984881786
  p-value: 0.4638671607940195
  Break point (0-based index): 0


In [19]:
from statsmodels.tsa.ardl import ARDL

# Example ARDL with one lag for each regressor
ardl_model = ARDL(endog=y, exog=X[['lnHC', 'lnL', 'lnK', 'lnGINI', 'lnINF']], lags=10, order={'lnHC': 1, 'lnL': 1, 'lnK': 1, 'lnGINI': 1, 'lnINF': 1})
ardl_result = ardl_model.fit()
print(ardl_result.summary())

                                 ARDL Model Results                                
Dep. Variable:                         lnY   No. Observations:                   55
Model:             ARDL(10, 1, 1, 1, 1, 1)   Log Likelihood                 136.276
Method:                    Conditional MLE   S.D. of innovations              0.012
Date:                     Fri, 07 Feb 2025   AIC                           -228.551
Time:                             10:31:43   BIC                           -188.805
Sample:                                 10   HQIC                          -213.734
                                        55                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6892      1.983      2.869      0.008       1.596       9.782
lnY.L1         0.6187      0.191      3.232      0.004       0.224       1.014
lnY.L2      

c:\Anaconda3\envs\ds_experiments\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


In [22]:
from statsmodels.tsa.ardl import ARDL

# Example ARDL with one lag for each regressor
ardl_model = ARDL(endog=y, exog=X[['lnHC', 'lnL', 'lnK', 'lnGINI', 'lnINF']], lags=[3], order={'lnHC': 1, 'lnL': 1, 'lnK': 1, 'lnGINI': 1, 'lnINF': 1})
ardl_result = ardl_model.fit()
print(ardl_result.summary())

                                ARDL Model Results                                
Dep. Variable:                        lnY   No. Observations:                   55
Model:             ARDL(3, 1, 1, 1, 1, 1)   Log Likelihood                 126.743
Method:                   Conditional MLE   S.D. of innovations              0.021
Date:                    Fri, 07 Feb 2025   AIC                           -227.487
Time:                            10:34:09   BIC                           -202.121
Sample:                                 3   HQIC                          -217.762
                                       55                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6108      1.762      2.617      0.012       1.049       8.172
lnY.L3         0.1296      0.142      0.913      0.367      -0.157       0.416
lnHC.L0       -3.469

c:\Anaconda3\envs\ds_experiments\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


In [23]:
from statsmodels.tsa.ardl import ARDL

# Example ARDL with one lag for each regressor
ardl_model = ARDL(endog=y, exog=X[['lnHC', 'lnL', 'lnK', 'lnGINI', 'lnINF']], lags=[3], order={'lnHC': 1, 'lnL': 1, 'lnK': [1], 'lnGINI': 1, 'lnINF': 1})
ardl_result = ardl_model.fit()
print(ardl_result.summary())

                                ARDL Model Results                                
Dep. Variable:                        lnY   No. Observations:                   55
Model:             ARDL(3, 1, 1, 1, 1, 1)   Log Likelihood                 117.014
Method:                   Conditional MLE   S.D. of innovations              0.025
Date:                    Fri, 07 Feb 2025   AIC                           -210.028
Time:                            10:34:42   BIC                           -186.613
Sample:                                 3   HQIC                          -201.052
                                       55                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.8988      2.068      2.853      0.007       1.723      10.074
lnY.L3         0.0747      0.168      0.444      0.659      -0.265       0.415
lnHC.L0        4.383

c:\Anaconda3\envs\ds_experiments\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
